In [ ]:

# loading the required libraries needed for data analysis
try:
    import pandas as pd
    import numpy as np
    from mplsoccer import Sbopen
    from statsbombpy import sb
except:
    %pip install mplsoccer
    %pip install statsbombpy
    import pandas as pd
    import numpy as np
    from mplsoccer import Sbopen
    from statsbombpy import sb

In [ ]:
# loading statsbomb to extract data
sbo=Sbopen()

In [ ]:
#Load the competition file
competitions = sbo.competition()

In [ ]:
competitions.head()

In [ ]:
# filtering La Liga matches from the entire data set
laLiga = competitions[competitions['competition_name']=='La Liga'].reset_index()

In [ ]:
# frinting the first 5 rows of the LA Liga data
laLiga.head()

In [ ]:
# extracting the competition ID for La Laiga
comp_id = laLiga['competition_id'][0]

In [ ]:
# filtering the years Messi played for Barcelona
messiYears = list(range(2004,2022))

In [ ]:
# extracting all the matches played by Messi
messiLaLiga =[]
messiSeasons =[]
for y in messiYears:
    for index, row in laLiga.iterrows():
        if(str(y) in row['season_name']):
            if(row['season_id'] in messiLaLiga):
                pass
            else:
                messiLaLiga.append(row['season_id'])
                messiSeasons.append(y)



In [ ]:
# the competition ID's where messi has played
messiLaLiga

In [ ]:
# the Years where Messi has played
messiSeasons

In [ ]:
# extracting all the data with the given years and competition ID
temp = sbo.match(competition_id=comp_id,season_id=1)

In [ ]:
#Load the list of matches for this competition
col = temp.columns.tolist()
matches = pd.DataFrame(columns=col)
count = 0
for i in messiLaLiga:
    match = sbo.match(competition_id=comp_id,season_id=i)
    match['season'] = messiSeasons[count]
    count = count+1
    matches = pd.concat([matches, match])

In [ ]:
matches.reset_index()

In [ ]:
# extracting the column names
col = matches.columns

In [ ]:
# filtering the columns needed for analysis
colKeep = ['match_id', 'match_date', 'match_week', 'season', 'home_score', 'away_score', 'home_team_name', 'home_team_managers_name'
           , 'away_team_name', 'away_team_managers_name', 'stadium_name', 'referee_name']

In [ ]:
# deleting the remaining columns from the data
colDel = [x for x in col if x not in colKeep]

In [ ]:
matches = matches.drop(colDel,axis=1)

In [ ]:
# filtering Barcelona's matches from the dataset
barca = matches[(matches['home_team_name']=='Barcelona') | (matches['away_team_name'] == 'Barcelona')]

In [ ]:
barca


In [ ]:
# extracting the home team manager, away team manager, home team, away team , stadium, and refree
manager=[]
oppManager=[]
homeTeam = []
awayTeam = []
stadium =[]
referee=[]
for index,row in barca.iterrows():
    if((type(row['home_team_managers_name'])==str)):
        if(row['home_team_name']=='Barcelona'):
            manager.append(row['home_team_managers_name'])
        else:
            manager.append(row['away_team_managers_name'])
    else:
        manager.append('')
    if((type(row['home_team_managers_name'])==str)):
        if(row['home_team_name']=='Barcelona'):
            oppManager.append(row['away_team_managers_name'])
        else:
            oppManager.append(row['home_team_managers_name'])
    else:
        oppManager.append('')


In [ ]:
manager

In [ ]:
# extracting manager information
for index, row in barca.iterrows():
    if(row['home_team_name']=='Barcelona'):
        row['home_team_managers_name'] = manager[index]
        row['away_team_managers_name'] = oppManager[index]
    else:
        row['home_team_managers_name'] = oppManager[index]
        row['away_team_managers_name'] = manager[index]


In [ ]:
# imputing missing values of manager
for index,row in matches.iterrows():
    if(row['home_team_name']=='Barcelona'):
        if(row['home_team_managers_name']==''):
            if(manager[index-1] != '' and manager[index+1]!='' and manager[index-1]==manager[index+1]):
                manager[index] = manager[index-1]
    else:
        if(row['away_team_managers_name']==''):
            if(manager[index-1] != '' and manager[index+1]!='' and manager[index-1]==manager[index+1]):
                manager[index] = manager[index-1]


In [ ]:
for index, row in barca.iterrows():
    if(row['home_team_name']=='Barcelona'):
        row['home_team_managers_name'] = manager
        row['away_team_managers_name'] = oppManager
    else:
        row['home_team_managers_name'] = oppManager
        row['away_team_managers_name'] = manager

In [ ]:
barca[barca['away_team_managers_name']=='']

In [ ]:
barca.head(1)

In [ ]:
# subsetting the dataframe with the needed columns
barca=barca[['season','match_id','match_week','match_date','home_score','away_score','home_team_name','home_team_managers_name','away_team_name','away_team_managers_name','stadium_name','referee_name']]

In [ ]:
# converting Barcelonas's match ID to list
match_id = barca['match_id'].to_list()

In [ ]:
# extrating the starting 11 players of all the Barcelona Match
temp = sbo.lineup(match_id=68314)[sbo.lineup(match_id=68314)['team_name']=='Barcelona']

In [ ]:
# extracting all the matched played by Messi
a =temp[temp['player_nickname']=='Lionel Messi']['match_id']

In [ ]:
#creating list of match ids in which messi played
messiMatchID =[]
for i in match_id:
    lineup = sbo.lineup(match_id=i)
    mid = lineup[lineup['player_nickname']=='Lionel Messi']['match_id'].to_list()
    if(len(mid)!=0):
        messiMatchID.append(mid[0])

In [ ]:
# extrating all the Info related to Messi's matches
messiMatches = barca[barca['match_id'].isin(messiMatchID)]

In [ ]:
messiMatches

In [ ]:
# extracting the opponents info for the matches played by Messi
opponent =[]
for index,row in messiMatches.iterrows():
    if(row['home_team_name']=='Barcelona'):
        opponent.append(row['away_team_name'])
    elif(row['away_team_name']=='Barcelona'):
        opponent.append(row['home_team_name'])

messiMatches['opponent'] = opponent

In [ ]:
messiMatches.to_csv('messiMatches.csv',index=False)

In [ ]:
# grouping the matches based on the season
messiMatchesG = messiMatches.groupby('season').agg(lambda x: list(x)).reset_index()

In [ ]:
lineup = sb.lineups(match_id=68314)['Barcelona']

In [ ]:
# extracting Messi's player ID from the lineup
messiID = lineup[lineup['player_nickname']=='Lionel Messi']['player_id'][1]

In [ ]:
messiID = float(messiID)

In [ ]:
messiID

In [ ]:
# creatign a temp variable to view events for the particular season
tempE = sb.competition_events(
            country="Spain",
            division= "La Liga",
            season="1973/1974"
        )

In [ ]:
# iterating through all the seasons and creating a data frame of all the events related to Messi
col = tempE.columns.tolist()
eventsMessi = pd.DataFrame(columns=col)
i=0
for season in laLiga['season_name']:
    if(season!='1973/1974'):
        event = sb.competition_events(
            country="Spain",
            division= "La Liga",
            season=season
        )
        event['season'] = season
        eventsMessi = pd.concat([eventsMessi, event[event['player_id']==messiID]])
    i=i+1

In [ ]:
eventsMessi = eventsMessi.reset_index()

In [ ]:
#separating x and y locations
eventsMessi[['x_start','y_start']] = eventsMessi['location'].apply(pd.Series)
eventsMessi[['x_end','y_end']] = eventsMessi['pass_end_location'].apply(pd.Series)
eventsMessi[['x_carry_end','y_carry_end']] = eventsMessi['carry_end_location'].apply(pd.Series)

In [ ]:
eventsMessi = eventsMessi.drop(['level_0'],axis=1)

In [ ]:
eventsMessi.to_csv('eventsMessi.csv',index=False)